# Generación de textos con Redes Neuronales

En este NoteBook se creará una red que pueda generar texto.  Se verá como lo hacer caracter por caracter.  En el siguiente enlace se encuentra un artículo interesante sobre esto: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Se ha organizado el proceso en "pasos" para que fácilmente se pueda utilizar con cualquier conjunto de datos

In [ ]:
# SOLO PARA USUARIOS DE GOOGLE COLLAB
%tensorflow_version 2.x

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# IGNORAR EL CONTENIDO DE ESTA CELDA
# tf.compat.v1.disable_eager_execution()

In [ ]:
tf.__version__

## Paso 1: Los Datos

Se puede bajar cualquier texto en forma gratuita desde este enlace:   www.gutenberg.org/

Para este ejercicio se han escogido todas las obras de Shakespear. Los datos ya han sido descargados.  Esta decisión se basó en dos razones:

1. Es un cuerpo enorme de texto, generalmente se recomiento que se tenga una fuente de al menos 1 millón de caracteres para lograr una generación de texto realista.

2. Tiene un estilo muy distinctivo.  Como el texto está en inglés antiguo, y está formateado en el estilo de una obra de teatro, será muy obvio si el modelo puede producir resultados similares.

In [ ]:
direccion_archivo = 'shakespeare.txt'

In [ ]:
texto = open(direccion_archivo, 'r').read()

In [ ]:
print(texto[:500])

### Ver cuáles son los caracteres únicos

In [ ]:
# Los caracteres únicos en el archivo
vocabulario = sorted(set(texto))
print(vocabulario)
len(vocabulario)  #importante tener en mente para trabajar la capa Dense

## Paso 2: Procesamiento de Texto

### Vectorización de Texto

Sabemos que una red neuronal no puede recibir datos en cadenas, es necesario asignar un número a cada caracter.  Se crearán dos diccionarios que puedan ir de índice numérico a caracter, y de caracter a índice numérico.

In [ ]:
caract_a_indice = {u:i for i, u in enumerate(vocabulario)}

In [ ]:
caract_a_indice

In [ ]:
indice_a_caract = np.array(vocabulario)

In [ ]:
indice_a_caract

In [ ]:
texto_codificado = np.array([caract_a_indice[c] for c in texto])

In [ ]:
texto_codificado

Ahora tenemos un mapeo que nos permite ir desde caracteres a numérico y viceversa.

In [ ]:
muestra = texto[:500]
muestra

In [ ]:
texto_codificado[:500]

## Paso 3: Crear Tandas

En general, lo que se trata de hacer es lograr que el modelo prediga el siguiente caracter de alta probabilidad, dados una secuencia histórica de caracteres.  

El usuario debe decidir qué tan larga va a ser esa secuencia histórica.  Una secuencia muy corta y no se tiene suficiente información (e.g. dados la letra "a", cuál es el siguiente caracter).  Una secuencia muy larga y el entrenamiento será muy largo y lo más probable es que *sobre ajuste* a caracteres secuenciales que son irrelevantes a caracters más lejanos.

Si bien no hay una selección de longitud de secuencia correcta, es importante considerar al texto mismo, que tan largo son las frases normales que tiene, y una idea razonable sobre qué caracteres/palabras son relevantes etre sí.

In [ ]:
print(texto[:500])

In [ ]:
linea = "From fairest creatures we desire increase"

In [ ]:
len(linea)

In [ ]:
parte_estrofa = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [ ]:
len(parte_estrofa)

### Secuencias de entrenamiento

El texto actual será la secuencia de texto desplazado hacia adelante en un caracter. Por ejemplo:


Secuencia Entrante: "Hola mi nom"
Secuencia Saliente: "ola mi nomb"


Se puede usar la función `tf.data.Dataset.from_tensor_slices` para  convertir un vector de texto a un flujo de indices de caracteres.

In [ ]:
# Viendo que una línea es aprox 40 caracteres y que Shakespeare
#   utiliza una rima, mas o menos, a cada 3 líneas, seleccionamos:
long_secuencia = 120  

In [ ]:
num_total_secuencias = len(texto)//(long_secuencia + 1)

In [ ]:
num_total_secuencias

In [ ]:
# Crear las secuencias de entrenamiento
conjunto_caract = tf.data.Dataset.from_tensor_slices(texto_codificado)
type(conjunto_caract)

In [ ]:
for i in conjunto_caract.take(500):
     print(indice_a_caract[i.numpy()])

El método de tandas convierte estas llamadas de caracteres individuales a secuencias que se pueden alimentar como una tanda.  Se utiliza long_secuencia + 1 debido a la indización empezando en cero.  Esto es lo que *drop_remainder* quiere decir: 


drop_remainder: (Opcional) Un escalar `tf.Tensor` de tipo `tf.bool`, que representa
    si la última tanda debe ser "botada" en caso tenga menos de 
    `batch_size` elementos; el comportamiento normal es no "botar" la tanda menor.
    
Esto es debido a la división de enteros que se hizo para calcular el número de secuencias...pueden quedar algunos residuos


In [ ]:
secuencias = conjunto_caract.batch(long_secuencia + 1, 
                                   drop_remainder = True)

Ahora que ya se tienen las secuencias, se ejecutarán los siguientes pasos para crear las secuencuas de texto meta:

1. Obtener la secuencia de texto entrante
2. Asignar la secuencia de texto meta como la secuencia de texto entrante, desplazada por un paso hacia adelante
3. Agruparlos como una tupla

In [ ]:
def crear_secuencias_meta(sec):
    texto_entrada = sec[:-1]
    texto_meta = sec[1:]
    return texto_entrada, texto_meta

In [ ]:
# El conjunto de datos final que se alimentará a la red
datos = secuencias.map(crear_secuencias_meta)

In [ ]:
for texto_entrada, texto_meta in datos.take(1):
    print(texto_entrada.numpy())
    print(''.join(indice_a_caract[texto_entrada.numpy()]))
    print('\n')
    print(texto_meta.numpy())
    # Hay espacio en blanco extra!
    print(''.join(indice_a_caract[texto_meta.numpy()]))

### Generar las tandas de entrenamiento

Ahora que se tienen las secuencias, se crearán las tandas.  Se "barajean" estas secuencias en un orden al azar, para que el modelo no se sobreajuste a cualquier sección de texto, pero que pueda generar caracteres dados cualquier texto "semilla".

In [ ]:
tamanio_tanda = 128

# Tamaño del espacio "Buffer" para barajear los datos con el fin 
#   de que no intente barajear toda la secuencie en memoria.  En 
#   vez, mantiene un  "buffer" en el cual barajea elementos
tamanio_buffer = 10000

datos = datos.shuffle(tamanio_buffer).batch(tamanio_tanda, 
                                            drop_remainder = True)

In [ ]:
datos

## Paso 4: Crear el Modelo

Se usará un modelo basado en LSTM con unas características extra, incluyendo una capa de incrustación "embedding" para empezar, y **dos** capas LSTM layers. Esta arquitectura de modelo se basa en [DeepMoji](https://deepmoji.mit.edu/) y la fuente original del código puede ser encontrada [aquí](https://github.com/bfelbo/DeepMoji).

La capa de incrustación servirá como la capa de entrada.  Escencialmente, esta crea una tabla de consulta que mapea los índices numéricos de cada caracter a un vector con "dim_incrust" número de dimensiones.  Como es de imaginar, entre más grande este tamaño de incrustación, más complejo el entrenamiento.  Esto es similar a la idea detrás de word2vec, donde las palabras se mapean a algún espacio n-dimensional. Hacer la incrustación antes de alimentar directamente al LSTM, generalmente conlleva a resultados mas realistas.

In [ ]:
# Longitud del vocabulario en caracteres
long_vocab = len(vocabulario)

# Dimensionamiento de la incrustación.  Se trata de que sea de
#    orden aproximado a long_vocab.  No es deseable que sea mucho 
#    más grande ya que el incremento en dimensiones afecta el 
#    tiempo de ejecución
dim_incrust = 64

# Número de unidades RNN
neuronas_rnn = 1024

Ahora se creará una función que se adapte fácilmente a variables diferentes como se ha mostrado arriba.

In [ ]:
from tensorflow.keras.models import Sequential

# se puede "jugar" con todo tipo de capas
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, GRU

# para este ejemplo solo usaremos Dense, Embedding, GRU

### Configurar la función de pérdida

Para la pérdida se utilizará *sparse categorical crossentropy*, que se puede importar de Keras.  Se selecciona esta debido a que las etiquetas están "one hot encoded"

También se dejará como logits = True, ya que este parámetro se refiere a si las etiquetas están, o no, "one hot encoded" 


In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
help(sparse_categorical_crossentropy)

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [ ]:
# Debido a que "sparse_categorical_crossentropy" tiene por default
#   "logits = False", se necesita una forma de cambiarlo.  Esto se 
#   hace envolviendo o poniendo un "wrapper" al método 

def perdida_categ_escasa(y_real,y_pred):
  return sparse_categorical_crossentropy(y_real, y_pred, 
                                         from_logits = True)

In [ ]:
def crear_modelo(tamanio_vocab, dim_incrust, neuronas_rnn, 
                 tamanio_tanda):
    
    modelo = Sequential()
    modelo.add(Embedding(tamanio_vocab, dim_incrust, 
                         batch_input_shape=[tamanio_tanda, None]))
    modelo.add(GRU(neuronas_rnn, return_sequences = True,
                   stateful = True, 
                   recurrent_initializer = 'glorot_uniform'))
    
    # Capa Final Densa para Predecir
    modelo.add(Dense(long_vocab))
    modelo.compile(optimizer = 'adam', loss = perdida_categ_escasa) 
    return modelo

In [ ]:
modelo = crear_modelo(long_vocab, dim_incrust,
                      neuronas_rnn, tamanio_tanda)

In [ ]:
modelo.summary()

## Paso 5: Entrenar el modelo

Antes de desperdiciar mucho tiempo con el modelo, se verifica que todo funcione bien.  Se le alimentará una tanda para asegurar que el modelo predice caracteres al azar, sin entrenar.

In [ ]:
for tanda_muestra_entrada, tanda_muestra_meta in datos.take(1):

  # Prededir a partir de una tanda al azar
  tanda_muestra_predicciones = modelo(tanda_muestra_entrada)

  # Desplegsar las dimensiones de las predicciones
  print(tanda_muestra_predicciones.shape, 
        " <=== (tamanio_tanda, long_secuencia, long_vocab)")


In [ ]:
tanda_muestra_predicciones

In [ ]:
indices_muestreados = tf.random.categorical(tanda_muestra_predicciones[0], 
                                            num_samples = 1)

In [ ]:
indices_muestreados

In [ ]:
# Reformatear para que no sea una lista de listas
indices_muestreados = tf.squeeze(indices_muestreados, 
                                 axis = -1).numpy()

In [ ]:
indices_muestreados

In [ ]:
print("Dado la secuencia de entrada: \n")
print("".join(indice_a_caract[tanda_muestra_entrada[0]]))
print('\n')
print("Predicciones del siguiente caracter: \n")
print("".join(indice_a_caract[indices_muestreados ]))


Luego de confirmar las dimensiones, se procede a entrenar la red!

In [ ]:
epocas = 30

In [ ]:
modelo.fit(datos, epochs = epocas)

## Paso 6: Generar texto

Como está ahorita, el modelo solo espera 128 secuencias a la vez.  Se puede crear un modelo que solo espere un tamanio_tanda = 1.  Se puede crear un modelo con este tamanio de tanda, y luego cargar los pesos que se han guardado.  Luego se invoca *.build()* sobre el modelo:

In [ ]:
modelo.save('shakespeare_gen.h5') 

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
modelo = crear_modelo(long_vocab, dim_incrust, 
                      neuronas_rnn, tamanio_tanda=1)

modelo.load_weights('shakespeare_gen.h5')

modelo.build(tf.TensorShape([1, None]))


In [ ]:
modelo.summary()

In [ ]:
def generar_texto(modelo, semilla_inicial, num_caract = 500, temp = 1.0):
  '''
  modelo: Modelo entrenado para Generar Texto
  
  semilla_inicial: Texto en formato cadena "string" a usar como semilla
  num_caract: Número de caracteres a generar

  La idea básica de esta función es la de tomar un texto semilla,
  formatearlo para que quede en la forma correcta para nuestra red,
  luego pasar la secuencia por una iteración conforme se le vayan
  agregando los caracteres predichos.  Parecido a lo que se hace
  con RNNs y series de tiempo.
  '''

  # Número de caracteres a generar
  num_generar = num_caract

  # Vectorización del texto semilla
  eval_entrada = [caract_a_indice[s] for s in semilla_inicial]

  # Expander para llegar a la forma requerida de tanda
  eval_entrada = tf.expand_dims(eval_entrada, 0)

  # Lista vacía para acumular el texto generado
  texto_generado = []

  # La "temperatura" afecta la aleatoriedad en el texto resultante
  # El término es derivado de entropía/termodinámica.
  # La "temperatura" se utiliza para afectar la probabilidad de
  #    los siguientes caracteres.
  # Temperatura mayor == menos sorprendente/ más esperado
  # Temperatura meno == más sorprendente / menos esperado
 
  temperatura = temp

  # Recordar que aquí tamanio_tanda == 1
  modelo.reset_states()

  for i in range(num_generar):

      # Generar Predicciones
      predicciones = modelo(eval_entrada)

      # Eliminar la dimensión de la forma de las tandas
      predicciones = tf.squeeze(predicciones, 0)

      # Usar una distribución categórica para escoger el
      #   siguiente caracter
      predicciones = predicciones / temperatura
      id_predicho = tf.random.categorical(predicciones, 
                                          num_samples = 1)[-1,0].numpy()

      # Pasar el caracter predicho para la siguiente entrada
      eval_entrada = tf.expand_dims([id_predicho], 0)

      # Transformar de vuelta a una letra
      texto_generado.append(indice_a_caract[id_predicho])

  return (semilla_inicial + ''.join(texto_generado))

In [ ]:
print(generar_texto(modelo, "flower", num_caract = 1000))